# Testing the PVG implementation

In [18]:
from math import sqrt

import torch
from torch.distributions import Categorical

from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader
from torch_geometric.data import Batch, Data
from torch_geometric.utils import to_networkx

import networkx as nx

from einops import rearrange

import plotly.graph_objects as go
import plotly.express as px

from pvg.graph_isomorphism import GraphIsomorphismData, GraphIsomorphismDataset
from pvg.parameters import Parameters
from pvg.graph_isomorphism import (
    GraphIsomorphismProver,
    GraphIsomorphismVerifier,
    GraphIsomorphismScenario,
)
from pvg.utils.torch_modules import PairedGaussianNoise
from pvg.extra.test_solo_gi_agents import GraphIsomorphismSoloProver

In [19]:
params = Parameters(
    scenario="graph-isomorphism",
    trainer="default",
    dataset="er1000",
    max_message_rounds=1,
    graph_isomorphism=dict(prover=dict(noise_sigma=1.0)),
)

In [20]:
dataset = TUDataset(root='/tmp/ENZYMES', name='ENZYMES', use_node_attr=True)

In [21]:
noise = PairedGaussianNoise(sigma=1)
noise

PairedGaussianNoise(sigma=1, pair_dim=0, train_sigma=False)

In [22]:
x = torch.tensor([[1, 2, 3], [4, 5, 6]])
noise(x)

tensor([[-0.3322,  3.9333,  1.6854],
        [-1.3290,  9.8333,  3.3709]])

In [23]:
y = torch.tensor([[1, 2, 3], [1, 2, 3]])
noise(y)

tensor([[ 0.9627, -3.6192,  3.1896],
        [ 0.9627, -3.6192,  3.1896]])

In [24]:
prover = GraphIsomorphismSoloProver(params, d_decider=16, device="cpu")
prover

GraphIsomorphismSoloProver(
  (gnn): SequentialKwargs(
    (input): TensorDictize(
      (module): Linear(in_features=1, out_features=16, bias=True)
    )
    (ReLU_0): TensorDictize(
      (module): ReLU(inplace=True)
    )
    (GNN_layer_0): GIN(
      (mlp): Sequential(
        (0): Linear(in_features=16, out_features=64, bias=True)
        (1): ReLU(inplace=True)
        (2): Linear(in_features=64, out_features=16, bias=True)
      )
    )
    (ReLU_1): TensorDictize(
      (module): ReLU(inplace=True)
    )
    (GNN_layer_1): GIN(
      (mlp): Sequential(
        (0): Linear(in_features=16, out_features=64, bias=True)
        (1): ReLU(inplace=True)
        (2): Linear(in_features=64, out_features=16, bias=True)
      )
    )
    (ReLU_2): TensorDictize(
      (module): ReLU(inplace=True)
    )
    (GNN_layer_2): GIN(
      (mlp): Sequential(
        (0): Linear(in_features=16, out_features=64, bias=True)
        (1): ReLU(inplace=True)
        (2): Linear(in_features=64, out_feat

In [25]:
dataset = GraphIsomorphismDataset(params)
dataset

GraphIsomorphismDataset(name='er1000', num_features=1, num_pairs=1000)

In [26]:
next(iter(dataset)).num_nodes

In [27]:
dataloader = DataLoader(
    dataset,
    batch_size=1,
    shuffle=False,
    follow_batch=["x_a", "x_b"],
)
dataloader

In [28]:
batch = next(iter(dataloader))
batch

GraphIsomorphismDataBatch(edge_index_a=[2, 10], edge_index_b=[2, 14], wl_score=[1], x_a=[7, 1], x_a_batch=[7], x_a_ptr=[2], x_b=[7, 1], x_b_batch=[7], x_b_ptr=[2])

In [29]:
prover(batch)

tensor([[-0.1930, -0.2751]], grad_fn=<AddmmBackward0>)

## Old

In [30]:
data1 = GraphIsomorphismData(
    edge_index_a=dataset[0].edge_index,
    edge_index_b=dataset[1].edge_index,
    num_nodes_a=dataset[0].size(0),
    num_nodes_b=dataset[1].size(0),
    d_features=params.max_message_rounds,
    y=1,
)
data2 = GraphIsomorphismData(
    edge_index_a=dataset[2].edge_index,
    edge_index_b=dataset[3].edge_index,
    num_nodes_a=dataset[2].size(0),
    num_nodes_b=dataset[3].size(0),
    d_features=params.max_message_rounds,
    y=1,
)
data1

TypeError: GraphIsomorphismData.__init__() got an unexpected keyword argument 'y'

In [ ]:
loader = DataLoader([data1, data2], batch_size=2, follow_batch=["x_a", "x_b"])
loader

In [ ]:
batch = next(iter(loader))
batch

GraphIsomorphismDataBatch(y=[2], edge_index_a=[2, 260], edge_index_b=[2, 192], x_a=[62, 8], x_a_batch=[62], x_a_ptr=[3], x_b=[47, 8], x_b_batch=[47], x_b_ptr=[3])

In [ ]:
torch.bincount(batch.x_a_batch)

tensor([37, 25])

In [ ]:
torch.cumsum(torch.bincount(batch.x_a_batch), dim=0) - torch.bincount(batch.x_a_batch)

tensor([ 0, 37])

In [ ]:
torch.bincount(batch.x_a_batch).max().item()

37

In [ ]:
isinstance(batch, Batch)

True

In [ ]:
scenario = GraphIsomorphismScenario(params, "cpu")
scenario

In [ ]:
rollout = scenario.rollout(batch)

In [ ]:
rollout.visualise()